In [1]:
HEAD_PATH = './'
DATA_PATH = HEAD_PATH + "data/stocks/"
SAVE_PATH = HEAD_PATH + "ckpt/"
TEMP_PATH = SAVE_PATH + "tmp pkl/"
OUTPUT_PATH = HEAD_PATH + 'output'

- Welcome to Quantitative and High Frequency Trading Training Program


In [2]:
from stock_helper import *
from stock_stats import *
from imp import reload  
import stock_helper
import stock_stats
reload(stock_helper)
reload(stock_stats)

<module 'stock_stats' from '/Users/junsu/Dropbox/workspace/stock_official/stock_stats.py'>

In [3]:
get_signal_pnl

<function stock_helper.get_signal_pnl(file, product, signal_name, thre_mat, reverse=1, tranct=0.00011, max_spread=0.61, tranct_ratio=True, HEAD_PATH='d:/intern', SAVE_PATH='d:/intern', atr_filter=0, rebate=0)>

In [4]:
import warnings
 
warnings.filterwarnings('ignore')

- number of cores, set this number according to your computer

In [5]:
#CORE_NUM = int(os.environ['NUMBER_OF_PROCESSORS'])
CORE_NUM =11

- sample product, you can change to your specific product symbol

In [6]:
product_list = ["600519", "000858", "000568", "600809", "002304"]

In [7]:
import dask
from dask import compute, delayed

- sample period

In [8]:
period = 4096


In [9]:
all_dates = os.listdir(DATA_PATH + product_list[0])
all_dates.sort()
all_dates = np.array(all_dates)
n_days = len(all_dates)
n_days


11

- choose an example day

In [10]:
train_sample = all_dates<"2017"

test_sample = all_dates>"2017"


- at last we plot the selected pnl

- now we can see that the number of signals are not very big
- and the quality is not so good
- we can propose a systematic method to generate a large number of signals then
- we can choose good ones from them

- for example, we can first generate some signals about volatility without direction
- there are some ideas about it

- standard deviation
- range of price
- ratio of volume to turnover

- then we can generate some signals with direction
- actually all of our previous signals have direction
- but some of them are good and some of them are bad
- we can focus on some good and simple ones, such as

- normalized return
- rsi
- dbook

- let's try to generate range signals first

In [11]:
## calculate rolling standard deviation
from collections import OrderedDict
class foctor_std_period(factor_template):
    factor_name = "std.period"
    
    params = OrderedDict([
        ("period", np.power(2, range(12,13)))
    ])
    
    def formula(self, data, period):
        return np.sqrt(fast_roll_var(data["wpr"], period))

In [12]:
## calcualte rolling variance
def fast_roll_var(x, period):
    x_ma = cum(x,period)/period ## roling first moment
    x2 = x*x
    x2_ma = cum(x2,period)/period ## rolling second moment
    var_x = x2_ma-x_ma*x_ma ## rolling variance
    return(var_x)

In [13]:
x20 = foctor_std_period()
for product in product_list:
    create_signal_path(x20, product, SAVE_PATH);

./ckpt//tmp pkl/600519/std.4096
./ckpt//tmp pkl/000858/std.4096
./ckpt//tmp pkl/000568/std.4096
./ckpt//tmp pkl/600809/std.4096
./ckpt//tmp pkl/002304/std.4096


In [15]:
%%time
for product in product_list:
    file_list = list(map(lambda x: DATA_PATH+product+"/"+x, sorted(os.listdir(DATA_PATH + product))))
    parLapply(CORE_NUM, file_list, build_composite_signal,signal_list=x20, product=product, HEAD_PATH=SAVE_PATH, n=8)
    
# Wall time: 1min 47s


CPU times: user 198 ms, sys: 224 ms, total: 422 ms
Wall time: 25.7 s


In [16]:
## rolling price range
from collections import OrderedDict
class foctor_range_period(factor_template):
    factor_name = "range.period"
    
    params = OrderedDict([
        ("period", np.power(2, range(12,13)))
    ])
    
    def formula(self, data, period):
        return data["max."+str(period)]-data["min."+str(period)]

In [17]:
x21 = foctor_range_period()
for product in product_list:
    create_signal_path(x21, product, SAVE_PATH);

./ckpt//tmp pkl/600519/range.4096
./ckpt//tmp pkl/000858/range.4096
./ckpt//tmp pkl/000568/range.4096
./ckpt//tmp pkl/600809/range.4096
./ckpt//tmp pkl/002304/range.4096


In [18]:
%%time
for product in product_list:
    file_list = list(map(lambda x: DATA_PATH+product+"/"+x, sorted(os.listdir(DATA_PATH + product))))
    parLapply(CORE_NUM, file_list, build_composite_signal,signal_list=x21, product=product, HEAD_PATH=SAVE_PATH, n=8)
    
## Wall time: 1min 8s


CPU times: user 144 ms, sys: 200 ms, total: 344 ms
Wall time: 23.3 s


In [19]:
## rolling trend index
from collections import OrderedDict
class foctor_trend_index_period(factor_template):
    factor_name = "trend.index.period"
    
    params = OrderedDict([
        ("period", np.power(2, range(12,13)))
    ])
    
    def formula(self, data, period):
        aa = zero_divide(abs(data["wpr"]-data["wpr"].shift(period)), data["max."+str(period)]-data["min."+str(period)])
        aa[0:period]=0
        return aa

In [20]:
x23 = foctor_trend_index_period()
for product in product_list:
    create_signal_path(x23, product, SAVE_PATH);


./ckpt//tmp pkl/600519/trend.index.4096
./ckpt//tmp pkl/000858/trend.index.4096
./ckpt//tmp pkl/000568/trend.index.4096
./ckpt//tmp pkl/600809/trend.index.4096
./ckpt//tmp pkl/002304/trend.index.4096


- Then we construt these range signals 

In [21]:
%%time
for product in product_list:
    file_list = list(map(lambda x: DATA_PATH+product+"/"+x, sorted(os.listdir(DATA_PATH + product))))
    parLapply(CORE_NUM, file_list, build_composite_signal,signal_list=x20, product=product, HEAD_PATH=SAVE_PATH, n=8)
;
# Wall time: 1min 11s


CPU times: user 196 ms, sys: 221 ms, total: 417 ms
Wall time: 25 s


''

In [22]:
%%time
for product in product_list:
    file_list = list(map(lambda x: DATA_PATH+product+"/"+x, sorted(os.listdir(DATA_PATH + product))))
    parLapply(CORE_NUM, file_list, build_composite_signal,signal_list=x23, product=product, HEAD_PATH=SAVE_PATH,n=8)
;

## Wall time: 1min 14s

CPU times: user 195 ms, sys: 210 ms, total: 405 ms
Wall time: 25.4 s


''

- Now we can combine directional signals and range signals together to become new direction signals

- For example, we have n directional signals, and m range signals, 
- then we can generate new n*m directional signals

In [23]:
## generate composite signal: trend signal * range signal = new trend signal
def construct_composite_signal(dire_signal, range_signal, period_list, product_list, HEAD_PATH):
    from collections import OrderedDict
    class foctor_xx_period(factor_template):
        factor_name = dire_signal+"."+range_signal+".period" ## name of new signal
        params = OrderedDict([
            ("period", period_list)
        ])
        def formula(self, data, period):
            return (data[dire_signal+"."+str(period)]*data[range_signal+"."+str(period)]).values ## calculation of new signal
    xx = foctor_xx_period()
    for product in product_list:
        create_signal_path(xx, product, HEAD_PATH)
        file_list = list(map(lambda x: DATA_PATH+product+"/"+x, sorted(os.listdir(DATA_PATH + product))))
        parLapply(CORE_NUM, file_list, build_composite_signal,signal_list=xx, product=product, HEAD_PATH=HEAD_PATH,n=8)

In [24]:
## generate composite signal: trend signal * range signal = new trend signal
def construct_composite_signal(dire_signal, range_signal, period_list, product_list, HEAD_PATH):
    from collections import OrderedDict
    class foctor_xx_period(factor_template):
        factor_name = dire_signal+"."+range_signal+".period" ## name of new signal
        params = OrderedDict([
            ("period", period_list)
        ])
        def formula(self, data, period):
            return (data[dire_signal+"."+str(period)]*data[range_signal+"."+str(period)]).values ## calculation of new signal
    xx = foctor_xx_period()
    for product in product_list:
        create_signal_path(xx, product, HEAD_PATH)
        file_list = list(map(lambda x: DATA_PATH+product+"/"+x, sorted(os.listdir(DATA_PATH + product))))
        parLapply(CORE_NUM, file_list, build_composite_signal,signal_list=xx, product=product, HEAD_PATH=HEAD_PATH,n=8)

In [25]:
dire_signal_list = ["nr", "dbook", "range.pos", "price.osci", "ma.dif.10", "kdj.k", "kdj.j"]
range_signal_list = ["range", "std", "trend.index"]
period_list = np.power(2, range(12,13))

In [26]:
%%time
for range_signal in range_signal_list:
    for dire_signal in dire_signal_list:
        construct_composite_signal(dire_signal, range_signal, period_list, product_list, SAVE_PATH)
##　Wall time: 25min 19s

./ckpt//tmp pkl/600519/nr.range.4096
./ckpt//tmp pkl/000858/nr.range.4096
./ckpt//tmp pkl/000568/nr.range.4096
./ckpt//tmp pkl/600809/nr.range.4096
./ckpt//tmp pkl/002304/nr.range.4096
./ckpt//tmp pkl/600519/dbook.range.4096
./ckpt//tmp pkl/000858/dbook.range.4096
./ckpt//tmp pkl/000568/dbook.range.4096
./ckpt//tmp pkl/600809/dbook.range.4096


KeyboardInterrupt: 

- Now we can combine signals of each day in to a vector
- here we use 4096 as period
- so to keep them independent, we choose 1 out of every 4096 points

In [33]:
period = 4096
os.makedirs(HEAD_PATH+"/all signal", exist_ok=True)
dire_signal_list = ["nr", "dbook", "range.pos", "price.osci", "ma.dif.10", "kdj.k", "kdj.j"]
range_signal_list = ["", "range", "std", "trend.index"]
all_signal_list = np.array([])
for range_signal in range_signal_list:
    for dire_signal in dire_signal_list:
        if len(range_signal)==0:
            signal_name = dire_signal
        else:
            signal_name = dire_signal+"."+range_signal
        all_signal_list = np.append(all_signal_list,signal_name)
all_period_signal = [signal+".4096" for signal in all_signal_list]
len(all_period_signal)
## 28

28

- we have 9 directional (trend) signals and 4 range signals
- so we have 36 composite signals 
- plus the 9 original ones we have 45 signals together


In [27]:
def moving_average(a, n=3) :
    ret_sum = np.cumsum(a, dtype=float)
    ret = a
    ret[n:] = (ret_sum[n:] - ret_sum[:-n])/n
    return ret

In [28]:
## parallel generate the distribution of a signal
def par_get_all_signal(signal_name, file_list, product, period, HEAD_PATH="d:/intern", SAVE_PATH="e:/intern", DATA_PATH="d:/intern"):
    n_files = len(file_list)
    all_signal = np.array([])
    for file in file_list:
        signal = load(HEAD_PATH+"/tmp pkl/"+product+"/"+signal_name+"/"+file) ## signal
        #moving_average(signal,period)
        chosen = (np.arange(len(signal))+1) % period==0
        all_signal = np.concatenate((all_signal, signal[chosen]), axis=0)
    save(all_signal, SAVE_PATH+"/all signal/"+product+"."+signal_name+".pkl")

In [34]:
%%time
for product in product_list:
    all_dates = os.listdir(DATA_PATH + product)
    parLapply(CORE_NUM, all_period_signal, par_get_all_signal, file_list=all_dates, product=product, period=4096, HEAD_PATH=SAVE_PATH,
             SAVE_PATH = "e:/intern", DATA_PATH="e:/intern");
## Wall time: 28.5 s

Wall time: 28.5 s


- then we can backtest our signals 
- we save the results

In [29]:
## this is to evalute the performance of a signal on a product
def evaluate_signal(signal, all_dates, product, min_pnl, min_num, 
                    CORE_NUM, HEAD_PATH="d:/intern", SIGNAL_PATH="d:/intern", period=4096, split_str="2018", 
                    atr_filter=0, save_path="signal result",reverse=0):
    signal_name = signal+"."+str(period) ## signal name, with period
    all_signal = load(SIGNAL_PATH+"/all signal/"+product+"."+signal_name+".pkl") ## get the distribution of the signal
    open_list = np.quantile(abs(all_signal), np.append(np.arange(0.991,0.999,0.001),np.arange(0.9991,0.9999,0.0001))) ## open threshold
    thre_list = []
    for cartesian in itertools.product(open_list, np.array([0.2, 0.4, 0.6, 0.8, 1.0])): ## close threshold
        thre_list.append((cartesian[0], -cartesian[0] * cartesian[1]))
    thre_list = np.array(thre_list)
    thre_mat = pd.DataFrame(data=OrderedDict([("open", thre_list[:, 0]), ("close", thre_list[:, 1])])) ## threshold matrix
    
    if reverse>=0: ## trending signal
        print("reverse=1")
        trend_signal_stat = get_signal_stat(signal_name, thre_mat, product, all_dates, CORE_NUM, split_str=split_str, reverse=1, 
                                    atr_filter=atr_filter, HEAD_PATH=HEAD_PATH, SAVE_PATH=SIGNAL_PATH)
    if reverse<=0: ## reversal signal
        print("reverse=-1")
        reverse_signal_stat = get_signal_stat(signal_name, thre_mat, product, all_dates, CORE_NUM, split_str=split_str, reverse=-1,
                                            atr_filter=atr_filter, HEAD_PATH=HEAD_PATH, SAVE_PATH=SIGNAL_PATH)
    if reverse==0: ## both trending and reversal
        stat_result = OrderedDict([("trend.signal.stat", trend_signal_stat), ("reverse.signal.stat", reverse_signal_stat)])    
        save(stat_result, HEAD_PATH+"/"+save_path+"/"+product+"."+signal_name+".pkl")
    elif reverse==1: ## just trend
        save(trend_signal_stat, HEAD_PATH+"/"+save_path+"/"+product+"."+signal_name+".trend.pkl")
    elif reverse==-1: ## just reversal
        save(reverse_signal_stat, HEAD_PATH+"/"+save_path+"/"+product+"."+signal_name+".reverse.pkl")

In [30]:
## get the statistics of backtesting a signal
from collections import OrderedDict
def get_signal_stat(signal_name, thre_mat, product, all_dates, CORE_NUM, split_str="2018", reverse=1, 
                     atr_filter=0, HEAD_PATH="d:/intern", SAVE_PATH="d:/intern"):
    train_sample = all_dates<split_str ## training samples
    test_sample = all_dates>split_str ## testing samples
    with dask.config.set(scheduler='processes', num_workers=CORE_NUM):
        f_par = functools.partial(get_signal_pnl, product=product, signal_name=signal_name, thre_mat=thre_mat,
                                 reverse=1,
                                HEAD_PATH=HEAD_PATH, SAVE_PATH=SAVE_PATH,atr_filter=atr_filter)
        train_result = compute([delayed(f_par)(file) for file in all_dates[train_sample]])[0] ## get training result
    train_stat = get_hft_summary(train_result, thre_mat) ## get training result statistics
    with dask.config.set(scheduler='processes', num_workers=CORE_NUM):
        f_par = functools.partial(get_signal_pnl, product=product, signal_name=signal_name, thre_mat=thre_mat,
                                 reverse=1,
                                HEAD_PATH=HEAD_PATH, SAVE_PATH=SAVE_PATH,atr_filter=atr_filter)
        test_result = compute([delayed(f_par)(file) for file in all_dates[test_sample]])[0] ## get testing result
    test_stat = get_hft_summary(test_result, thre_mat) ## get testing result statistics
    return OrderedDict([("train.stat", train_stat), ("test.stat", test_stat)])

In [31]:
import os
os.makedirs(HEAD_PATH+"/signal result atr/", exist_ok=True)

In [34]:
%%time
#SIGNAL_PATH = "e:/intern"
for product in product_list:
    all_dates = np.array(os.listdir(DATA_PATH + product))
    for signal in all_signal_list:
        print(product, signal)
        evaluate_signal(signal, all_dates, product, 0.001, 20, CORE_NUM, HEAD_PATH, SAVE_PATH,
                        period=4096, split_str="2017",
            atr_filter=0.02, save_path="signal result atr")

## Wall time: 14h 53min 10s

600519 nr


TypeError: bad operand type for abs(): 'collections.OrderedDict'

- we can write a function to get all the signals' performance

- we can get the result with atr and without atr separately

In [38]:
## get the signal performance
## including trend and reverse signals
def get_signal_performance_result(all_period_signal, signal_dire, period, product_list):
    trend_signal_result = pd.DataFrame(data=OrderedDict([("signal", all_period_signal), ("reverse",1),
                               ("num", 0), ("trainSharpe", 0), ("testSharpe", 0)]))
    reverse_signal_result = pd.DataFrame(data=OrderedDict([("signal", all_period_signal), ("reverse",-1),
                               ("num", 0), ("trainSharpe", 0), ("testSharpe", 0)]))
    n_signal = len(all_period_signal) ## number of all signals
    trend_train_sharpe = np.zeros(len(product_list))
    trend_test_sharpe = np.zeros(len(product_list))
    reverse_train_sharpe = np.zeros(len(product_list))
    reverse_test_sharpe = np.zeros(len(product_list))
    avg_pnl = 0.001
    for k in range(n_signal):
        signal_name = all_period_signal[k]
        i = 0
        for product in product_list:
            all_dates = np.array(os.listdir(DATA_PATH + product))
            train_sample = all_dates<"2019" 
            test_sample = all_dates>"2019"
            stat_result = load(HEAD_PATH+"/" + signal_dire +"/"+product+"."+signal_name+".pkl") ## statistics of signal over a product
            trend_signal_stat = stat_result['trend.signal.stat'] ## trending statistics
            
            if tuple(trend_signal_stat.keys())[0]=='train.stat':
                train_stat = trend_signal_stat["train.stat"]
                test_stat = trend_signal_stat["test.stat"]
                #good_strat = trend_signal_stat["good.strat"]
                good_strat = (train_stat["final.result"]["avg.ret"]>avg_pnl) & (train_stat["final.result"]["num"]>10)
                if sum(good_strat)>2:
                    train_stat = trend_signal_stat["train.stat"]
                    test_stat = trend_signal_stat["test.stat"]
                    train_pnl = train_stat["daily.pnl"][:, good_strat].sum(axis=1)/sum(good_strat)
                    #train_std = np.std(train_pnl)
                    #train_pnl = train_pnl/train_std
                    test_pnl = test_stat["daily.pnl"][:, good_strat].sum(axis=1)/sum(good_strat)
                    trend_train_sharpe[i] = sharpe(train_pnl)
                    trend_test_sharpe[i] = sharpe(test_pnl)
                    #print(product, "train sharpe ", sharpe(train_pnl), "test sharpe ", sharpe(test_pnl))
                    i = i+1
            if i>0: ## if there are any good products
                trend_signal_result.loc[k, ("signal", "num", "trainSharpe", "testSharpe")] = (signal_name, i,  np.mean(trend_train_sharpe[:i]),np.mean(trend_test_sharpe[:i]))
        i = 0
        for product in product_list:
            stat_result = load(HEAD_PATH+"/"+signal_dire+"/"+product+"."+signal_name+".pkl")
            reverse_signal_stat = stat_result['reverse.signal.stat']
            if tuple(reverse_signal_stat.keys())[0]=='train.stat':
                #good_strat = reverse_signal_stat["good.strat"]
                train_stat = reverse_signal_stat["train.stat"]
                train_pnl = train_stat["daily.pnl"][:, good_strat].sum(axis=1)/sum(good_strat)
                test_stat = reverse_signal_stat["test.stat"]
                good_strat = (train_stat["final.result"]["avg.ret"]>avg_pnl) & (train_stat["final.result"]["num"]>10)
                test_pnl = test_stat["daily.pnl"][:, good_strat].sum(axis=1)/sum(good_strat)
                if sum(good_strat)>2:
                    #train_std = np.std(train_pnl)
                    #train_pnl = train_pnl/train_std
                    reverse_train_sharpe[i] = sharpe(train_pnl)
                    reverse_test_sharpe[i] = sharpe(test_pnl)
                    i = i+1
            if i>0:
                
                reverse_signal_result.loc[k, ("signal","num", "trainSharpe", "testSharpe")] = (signal_name, i, np.mean(reverse_train_sharpe[:i]),np.mean(reverse_test_sharpe[:i]))
    return OrderedDict([("trend.signal.stat", trend_signal_result), 
                        ("reverse.signal.stat", reverse_signal_result)])


In [39]:
result_atr = get_signal_performance_result(all_period_signal, "signal result atr", 4096, product_list)

In [40]:
def get_single_signal_stat(result_atr, min_num=0, min_sharpe=0.5):
    good_trend = (result_atr["trend.signal.stat"]["num"]>min_num) & (result_atr["trend.signal.stat"]["trainSharpe"]>min_sharpe) & (result_atr["trend.signal.stat"]["testSharpe"]>min_sharpe)
    good_reverse = (result_atr["reverse.signal.stat"]["num"]>min_num) & (result_atr["reverse.signal.stat"]["trainSharpe"]>min_sharpe) & (result_atr["reverse.signal.stat"]["testSharpe"]>min_sharpe)
    print("with atr good signals: "+ str(sum(good_trend | good_reverse)))
    train_sharpe = np.mean(np.append(result_atr["trend.signal.stat"]["trainSharpe"][good_trend],
                      result_atr["reverse.signal.stat"]["trainSharpe"][good_reverse]))
    test_sharpe = np.mean(np.append(result_atr["trend.signal.stat"]["testSharpe"][good_trend],
                      result_atr["reverse.signal.stat"]["testSharpe"][good_reverse]))
    print("train sharpe: ", train_sharpe)
    print("test sharpe: ", test_sharpe)
    print(all_signal_list[good_trend])
    print(all_signal_list[good_reverse])


In [41]:
get_single_signal_stat(result_atr, min_num=0, min_sharpe=0)

# with atr good signals: 7
# train sharpe:  0.25031657008063096
# test sharpe:  0.3381809954027074
# ['nr' 'price.osci' 'kdj.k' 'kdj.j' 'nr.trend.index'
#  'range.pos.trend.index' 'price.osci.trend.index']
# []






with atr good signals: 7
train sharpe:  0.25031657008063096
test sharpe:  0.3381809954027074
['nr' 'price.osci' 'kdj.k' 'kdj.j' 'nr.trend.index'
 'range.pos.trend.index' 'price.osci.trend.index']
[]


In [42]:
get_single_signal_stat(result_atr, min_num=1, min_sharpe=0)

# with atr good signals: 6
# train sharpe:  0.232794205322825
# test sharpe:  0.335934275190708
# ['nr' 'price.osci' 'kdj.k' 'nr.trend.index' 'range.pos.trend.index'
#  'price.osci.trend.index']
# []





with atr good signals: 6
train sharpe:  0.232794205322825
test sharpe:  0.335934275190708
['nr' 'price.osci' 'kdj.k' 'nr.trend.index' 'range.pos.trend.index'
 'price.osci.trend.index']
[]


In [43]:
get_single_signal_stat(result_atr, min_num=2, min_sharpe=0)

# with atr good signals: 4
# train sharpe:  0.22417098533953134
# test sharpe:  0.2856700020721965
# ['kdj.k' 'nr.trend.index' 'range.pos.trend.index' 'price.osci.trend.index']
# []


with atr good signals: 4
train sharpe:  0.22417098533953134
test sharpe:  0.2856700020721965
['kdj.k' 'nr.trend.index' 'range.pos.trend.index' 'price.osci.trend.index']
[]


In [44]:
get_single_signal_stat(result_atr, min_num=3, min_sharpe=0)

# with atr good signals: 1
# train sharpe:  0.21218760742128087
# test sharpe:  0.487643784373875
# ['price.osci.trend.index']
# []



with atr good signals: 1
train sharpe:  0.21218760742128087
test sharpe:  0.487643784373875
['price.osci.trend.index']
[]


In [45]:
get_single_signal_stat(result_atr, min_num=4, min_sharpe=0)

# with atr good signals: 1
# train sharpe:  0.21218760742128087
# test sharpe:  0.487643784373875
# ['price.osci.trend.index']
# []


with atr good signals: 1
train sharpe:  0.21218760742128087
test sharpe:  0.487643784373875
['price.osci.trend.index']
[]


- we can see that their results are very similar

- Conclusion
- This week we have generated results for all of the 35 signals
- We have compared backtest result with and without atr
- we find that there is not much difference between them
- previously we found that for iorn with atr is better than without atr
- that's because we lower the threhsold for number of trades
- now we use 20 trades for training maybe for i there are not enough trades
- Anyway, currently there are too many signals and hard to tune parameters
- in the future, we can use fewer signals so that we can do the research more efficiently